In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import gc
import tensorflow as tf

from joblib import Parallel, delayed


from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, roc_auc_score, f1_score, plot_confusion_matrix
import lightgbm as lgb

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras import regularizers


from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression


from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import RepeatVector

from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, Flatten, Dense, MaxPooling1D, GlobalMaxPool1D, GlobalMaxPool2D, Conv2D, MaxPool2D
from tensorflow.keras.layers import Embedding, Dropout, Bidirectional, LSTM, GlobalMaxPool1D, Dense, GRU, Reshape, Concatenate, BatchNormalization
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [ ]:
from tensorflow.keras import backend as K
def rmspe_keras(y_true, y_pred):
    return K.sqrt(K.mean(K.square((y_true - y_pred) / y_true)))

In [ ]:
data_dir = '../input/optiver-realized-volatility-prediction/'
train = pd.read_csv(data_dir+'train.csv')
test = pd.read_csv(data_dir+ 'test.csv')

# Single Stock 0

In [ ]:
b0 = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0")
b0.head()

In [ ]:
# Only taking bid 1 and ask 1
cols= ['time_id','bid_price1', 'ask_price1']
df = b0[cols]

In [ ]:
# 10 minutes 
max_len = 600

In [ ]:
group = df.groupby('time_id')

In [ ]:
# max Seq len 
group.size().max()

In [ ]:
# prepare for 600 seconds so that all stock have same no of sequences
grouped_arr = []
for time , data in tqdm(group):
    emp = np.zeros((max_len, df.shape[1]))
    arr = data.values
    for ind in range(arr.shape[0]):
        emp[ind] = arr[ind]
    grouped_arr.append(emp)
filled_df = pd.DataFrame(np.vstack(grouped_arr)).replace(0, method='ffill')
filled_df.columns = df.columns

In [ ]:
# Prepare time series

group_by = 'time_id' # can be anything

feature_cols = ['bid_price1', 'ask_price1']
x_arr = filled_df.loc[:, feature_cols].values

#create new group having 600 sequences values
group = filled_df.groupby(group_by)

x_seq = [x_arr[v.values] for k, v in group.groups.items() ]

X = np.array(x_seq)
X.shape

In [ ]:
# Target 
y = train[train['stock_id'] ==0]['target'].values

In [ ]:
X.shape, y.shape

# LSTM 

In [ ]:
X_train, X_test = X[:2800], X[2800:]
y_train, y_test = y[:2800], y[2800:]

In [ ]:
book_input = tf.keras.layers.Input(shape = (600, 2))
book = tf.keras.layers.BatchNormalization()(book_input)
book = tf.keras.layers.LSTM(256, return_sequences=True)(book)
book = tf.keras.layers.Dropout(0.1)(book)
book = tf.keras.layers.LSTM(128)(book)
book = tf.keras.layers.Dense(128)(book)
out = tf.keras.layers.Dense(1, activation='linear')(book)

model = tf.keras.Model(inputs = book_input, outputs=out)
model.compile(loss=rmspe_keras, optimizer=tf.keras.optimizers.Adam(0.001))
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
preds = model.predict(X_test, batch_size=32)
preds =preds.flatten()
print(f"RMSPE on test set: {rmspe_keras(y_test, preds)}")

In [ ]:
preds_df = pd.concat([ pd.Series(y_test), pd.Series(preds)], axis=1)
preds_df.columns = ['target', 'predicted']
preds_df.plot(figsize=(12, 6), alpha=0.9)